In [ ]:
import os,tqdm,cv2,sys
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator

#取消像素限制
Image.MAX_IMAGE_PIXELS = None

## CV2操作

#### 简单操作

In [ ]:
def re(i,i2,o):
    if not os.path.exists(o):
        os.makedirs(o)
    for name in tqdm.tqdm(os.listdir(i)):
        img_path = os.path.join(i,name)
        img_path2 = os.path.join(i2,name.split('.')[0]+'.jpg')
        save_path = os.path.join(o,name.split('.')[0]+'.jpg')
        img = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
        h,w = img.shape
        img2 = cv2.imread(img_path2)
        img2 = cv2.resize(img2,(w,h))
        cv2.imwrite(save_path,img2)

i = r"D:\31890\Desktop\codefile\Utils\biaozhu"
i2 = r'D:\31890\Desktop\codefile\data\Train_data\Datasets\senescence\1\o'
o = r"D:\31890\Desktop\codefile\Utils\o2"
re(i,i2,o)

#### 灰度图像的滤波

In [ ]:
def image_filter(img_file,filter_file):
    '''     img_file:输入的灰度图片
            filter_file:滤波保存的图片        '''
    for name in tqdm.tqdm(os.listdir(img_file)):
        img_path = os.path.join(img_file,name)
        original_image = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
        # 均值滤波
        img_blur=cv2.blur(original_image,(3,3))
        save_path= os.path.join(filter_file,name.split('.')[0]+"Mean_"+".png")
        cv2.imwrite(save_path,img_blur)
        #方框滤波
        img_boxFilter1 = cv2.boxFilter(original_image, -1, (5, 5), normalize=True)
        save_path= os.path.join(filter_file,name.split('.')[0]+"box_"+'.png')
        cv2.imwrite(save_path,img_boxFilter1)
        #高斯滤波
        img_GaussianBlur= cv2.GaussianBlur(original_image, (5, 5), 0, 0)
        save_path= os.path.join(filter_file,name.split('.')[0]+"Gaussian_"+".png")
        cv2.imwrite(save_path,img_GaussianBlur)
        # 中值滤波
        img_medianBlur = cv2.medianBlur(original_image, 5)
        save_path= os.path.join(filter_file,name.split('.')[0]+"median_"+".png")
        cv2.imwrite(save_path,img_medianBlur)
        # 双边滤波
        img_bilateralFilter=cv2.bilateralFilter(original_image,50,100,100)
        save_path= os.path.join(filter_file,name.split('.')[0]+"Bilateral_"+'.png')
        cv2.imwrite(save_path,img_bilateralFilter)
        # 形态学滤波
        kernel = np.random.normal(0.01,1,size=(5,5))
        kernel = np.ones((3,3))
        img_mo = cv2.morphologyEx(original_image,cv2.MORPH_CLOSE,kernel)
        save_path= os.path.join(filter_file,name.split('.')[0]+"clsoe"+".png")
        cv2.imwrite(save_path,img_mo)
        kernel = np.ones((3,3))
        img_mo = cv2.morphologyEx(original_image,cv2.MORPH_OPEN,kernel)
        save_path= os.path.join(filter_file,name.split('.')[0]+"open"+".png")
        cv2.imwrite(save_path,img_mo)

In [ ]:
img_path = r"D:\31890\Desktop\1\10.jpg"
original_image = cv2.imread(img_path)
# 均值滤波
img_blur=cv2.blur(original_image,(8,8))
# img_blur=cv2.blur(img_blur,(8,12))

cv2.imwrite(r"D:\31890\Desktop\1\10_2.jpg",img_blur)

#### 图像去噪加图像增强

In [ ]:
def Denoising_enhancement(img_file,save_path):
    '''     img_file:输入图片的路径
            save_path:保存图片的路径      '''
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    for name in tqdm.tqdm(os.listdir(img_file)):
        image_name = os.path.splitext(name)[0]
        img = cv2.imread(os.path.join(img_file,name))
        # h参数调节过滤器强度。大的h值可以完美消除噪点，但同时也可以消除图像细节，较小的h值可以保留细节但也可以保留一些噪点
        h = 10
        # templateWindowSize用于计算权重的模板补丁的像素大小，为奇数，默认7
        templateWindowSize = 5
        # searchWindowSize窗口的像素大小，用于计算给定像素的加权平均值，为奇数，默认21
        searchWindowSize = 21
        # dst = cv2.fastNlMeansDenoising(img, None, h, h, templateWindowSize, searchWindowSize)
        #非局部均值滤波
        Deno = cv2.fastNlMeansDenoising(img,None, h, templateWindowSize, searchWindowSize)
        #高斯模糊
        img_GaussianBlur= cv2.GaussianBlur(Deno, (5, 5), 0, 0)
        #USM增强
        usm = cv2.addWeighted(img,1.5,img_GaussianBlur,-0.5,0)  
        #保存
        result_path = os.path.join(save_path,image_name + ".png")
        cv2.imwrite(result_path,usm)

In [ ]:
def cv2show(image,name="image1",h=1000,w=900):
    cv2.namedWindow(str(name), cv2.WINDOW_KEEPRATIO)
    # WINDOW_NORMAL	用户能够调节窗口大小
    # WINDOW_AUTOSIZE	根据图像大小显示窗口，大小固定
    # WINDOW_FREERATIO	调整图像。不考虑其比例
    # WINDOW_KEEPRATIO	保持图像比例
    cv2.resizeWindow(str(name), w ,h)
    cv2.imshow(str(name),image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

#### 图像梯度

In [ ]:
import cv2 
import numpy as np
 
# Scharr算子实现梯度计算
def Scharr_demo(image_array):

    # cv2.namedWindow('image_gradient-x', 0)
    # cv2.resizeWindow('image_gradient-x', 900 ,1000)
    # cv2.namedWindow('image_gradient-y', 0)
    # cv2.resizeWindow('image_gradient-y', 900 ,1000)    
    # cv2.namedWindow('iamg_', 0)
    # cv2.resizeWindow('iamg_', 900 ,1000)
    # cv2.namedWindow('img_GaussianBlur', 0)
    # cv2.resizeWindow('img_GaussianBlur', 900 ,1000)
    # cv2.namedWindow('image_gradient', 0)
    # cv2.resizeWindow('image_gradient', 900 ,1000)

    # x 方向梯度
    image_grad_x = cv2.Scharr(image_array, cv2.CV_32F, 1, 0)

    # y 方向梯度
    image_grad_y = cv2.Scharr(image_array, cv2.CV_32F, 0, 1)

    # 分别求绝对值并转化为8位的图像上，这样做方便显示
    image_gradx = cv2.convertScaleAbs(image_grad_x) 
    image_grady = cv2.convertScaleAbs(image_grad_y)

    # 显示两个方向图像
    # cv2.imshow("image_gradient-x", image_gradx)
    # cv2.imshow("image_gradient-y", image_grady)
    
    #两个方向梯度的叠加，权重各自一半
    # image_gradxy = cv2.addWeighted(image_gradx, 0.5, image_grady, 0.5, 0)
    # cv2.imshow("image_gradient", image_gradxy)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return image_grady
    

image_path = r'D:\software\Code\code-file\pytorch-template\test\1.png'
save_path = r'D:\software\Code\code-file\pytorch-template\test\2.png'
Scharr_demo(image_path,save_path)


#### 腐蚀


In [ ]:
image_path = r'D:\software\Code\code-file\pytorch-template\test2\1_grad_y.png'
save_path = r'D:\software\Code\code-file\pytorch-template\test2\1_grad_y_erode2.png'

def erode_demo(image_array):
    # 创建矩形结构元
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,2))
    # 腐蚀图像
    img_erode = cv2.erode(image_array, kernel,iterations=2)
    return img_erode

#### 膨胀

In [ ]:
image_path = r'D:\software\code\code-file\image\test\02-20190619-1200-0538_3_13.png'
def dilate_demo(image_path,save_path):
    img_src = cv2.imread(image_path, 0)
    # 创建结构元
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (8,8))
    # 腐蚀图像,
    img_dilate = cv2.dilate(img_src, kernel,iterations=5)
    # cv2.namedWindow('img_src', 0)
    # cv2.resizeWindow('img_src', 200, 200)
    # cv2.imshow("img_src",img_src)
    cv2.imwrite(save_path,img_dilate)

    # cv2.waitKey(0)

#### 形态学操作

In [ ]:
import os,tqdm,cv2,sys
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator

#取消像素限制
Image.MAX_IMAGE_PIXELS = None
image_path = r"D:\31890\Desktop\1\200.jpg"
save_path = r"D:\31890\Desktop\1\200_1.jpg"

def morphologyEx_demo(image_path,save_path):
    iamge = cv2.imread(image_path)
    # kernel = cv2.getStructuringElement(cv2.MORPH_CLOSE,(2,2))#3,40
    kernel = np.ones((4,4),np.uint8)

    # MORPH_ERODE 腐蚀 erode
    # MORPH_DILATE 膨胀 dilate
    # MORPH_OPEN 开运算（先腐蚀后膨胀）
    # MORPH_CLOSE 闭运算（先膨胀后腐蚀）
    # MORPH_GRADIENT 形态学梯度
    # MORPH_TOPHAT 顶帽运算
    # MORPH_BLACKHAT 底帽运算
    
    result  = cv2.morphologyEx(iamge,cv2.MORPH_CLOSE,kernel,iterations=2)
    cv2.imwrite(save_path,result)

morphologyEx_demo(image_path,save_path)

#### 滑条操作

In [ ]:
img_src = cv2.imread(r"D:\software\Code\code-file\image\Root_senescence\egnet_pre_cv2\open\2\02-20190619-1200-0538.png", 0)

def nothing(*args):
    pass
r, i = 1, 1
max_r, max_i = 20, 20

cv2.namedWindow('morphology', 0)
cv2.resizeWindow('morphology', 900 ,1000)
cv2.createTrackbar('r', 'morphology', r, max_r, nothing)
cv2.createTrackbar('i', 'morphology', i, max_i, nothing)

while True:
    r = cv2.getTrackbarPos('r', 'morphology')
    i = cv2.getTrackbarPos('i', 'morphology')
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2*r+1, 2*r+1))
    # 形态学操作
    # MORPH_ERODE 腐蚀 erode
    # MORPH_DILATE 膨胀 dilate
    # MORPH_OPEN 开运算（先腐蚀后膨胀）
    # MORPH_CLOSE 闭运算（先膨胀后腐蚀）
    # MORPH_GRADIENT 形态学梯度
    # MORPH_TOPHAT 顶帽运算
    # MORPH_BLACKHAT 底帽运算
    img_erode = cv2.morphologyEx(img_src,cv2.MORPH_OPEN, kernel, iterations=i)
    cv2.imshow("morphology", img_erode)
    cv2.waitKey(10)
    # cv2.imwrite(r'D:\software\Code\code-file\pytorch-template\test2\2_ERODE.png',img_erode)
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np

# 滑动条的回调函数，获取滑动条位置处的值
def empty(a):
    h_min = cv2.getTrackbarPos("Hue Min","TrackBars")
    h_max = cv2.getTrackbarPos("Hue Max", "TrackBars")
    s_min = cv2.getTrackbarPos("Sat Min", "TrackBars")
    s_max = cv2.getTrackbarPos("Sat Max", "TrackBars")
    v_min = cv2.getTrackbarPos("Val Min", "TrackBars")
    v_max = cv2.getTrackbarPos("Val Max", "TrackBars")
    # print(h_min, h_max, s_min, s_max, v_min, v_max)
    return h_min, h_max, s_min, s_max, v_min, v_max

path = r'D:\software\Code\code-file\pytorch-template\test\cmask_sub.png'
# 创建一个窗口，放置6个滑动条
cv2.namedWindow("TrackBars",0)
cv2.resizeWindow("TrackBars",320,240)
cv2.createTrackbar("Hue Min","TrackBars",0,179,empty)
cv2.createTrackbar("Hue Max","TrackBars",19,179,empty)
cv2.createTrackbar("Sat Min","TrackBars",110,255,empty)
cv2.createTrackbar("Sat Max","TrackBars",240,255,empty)
cv2.createTrackbar("Val Min","TrackBars",153,255,empty)
cv2.createTrackbar("Val Max","TrackBars",255,255,empty)

while True:
    img = cv2.imread(path)
    imgHSV = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    # 调用回调函数，获取滑动条的值
    h_min,h_max,s_min,s_max,v_min,v_max = empty(0)
    lower = np.array([h_min,s_min,v_min])
    upper = np.array([h_max,s_max,v_max])
    # 获得指定颜色范围内的掩码
    mask = cv2.inRange(imgHSV,lower,upper)
    # 对原图图像进行按位与的操作，掩码区域保留
    imgResult = cv2.bitwise_and(img,img,mask=mask)
   
    # cv2.imshow("Mask", mask)
    cv2.imshow("Result", imgResult)
    
    cv2.waitKey(1)

#### 图像相减

In [ ]:
image1_path = r'D:\software\Code\code-file\pytorch-template\test\1_b.png'
image2_path = r'D:\software\Code\code-file\pytorch-template\test\3_2.png'
save_path = r'D:\software\Code\code-file\pytorch-template\test\4.png'

def subtract_demo(image1_array,image2_array):

    image = image1_array - image2_array

    return image

subtract_demo()

In [ ]:
## 遍历相减

def ergodic_func(image1_path,target_path,save_path):
    image1 = cv2.imread(image1_path,cv2.IMREAD_GRAYSCALE)
    target = cv2.imread(target_path,cv2.IMREAD_GRAYSCALE)

    h,w = image1.shape
    for x in range(w):
        for y in range (w):
            if image1[x,y] !=  0:
                target[x,y]=0
    
    cv2.imwrite(save_path,target)


image1_path = r'D:\software\Code\code-file\pytorch-template\test2\1_grad_y_ERODE.png'
target_path = r'D:\software\Code\code-file\pytorch-template\test2\1_threshold.png'
save_path = r'D:\software\Code\code-file\pytorch-template\test2\2.png'
ergodic_func(image1_path,target_path,save_path)

#### 图像归一化

In [ ]:
import albumentations as A

transform =A.Compose([
    A.Normalize(mean=[0.4754358, 0.35509014, 0.282971],std=[0.16318515, 0.15616792, 0.15164918],max_pixel_value=255.0, always_apply=False)])

image_file = r'D:\31890\Pictures\image\Root_senescence\split_image_o\2'
save_file = r'D:\31890\Pictures\image\Root_senescence\split_image_norm\2'
if not os.path.exists(save_file):
    os.makedirs(save_file)
for name in os.listdir(image_file):
    iamge_Path = os.path.join(image_file,name)
    save_path  = os.path.join(save_file,name)
    image = cv2.imread(iamge_Path)
    transformed = transform(image=image)
    transformed_image = transformed['image']
    # cv2.imshow("a",transformed_image)
    print(transformed_image)
    # cv2.imshow("b",transformed_mask)
    cv2.waitKey(0)
    cv2.imwrite(save_path,transformed_image)

#### alpha通道

In [ ]:
img = cv2.imread(r'D:\software\code\code-file\T\image\mydata\my_data\imgs\train\1_4_6.jpg')

        # 使用cv.merge()函数添加alpha通道
zeros = np.ones(img.shape[:2], dtype=img.dtype) *255
result_BGR_alpha = cv2.merge([img, zeros])
print('原图的通道数为：{}'.format(img.shape[2]))
print('处理后的通道数为：{}'.format(result_BGR_alpha.shape[2]))

cv2.imwrite(r'D:\31890\Desktop\1\1.png',result_BGR_alpha)

#通道提取
cv2.imwrite(r'D:\31890\Desktop\1\1.png',result_BGR_alpha)

#### 获取直方图

In [ ]:
#BGR单通道的直方图
def BGR_signal_channel_hist_process(BGR_signal_file_one_channel,model_pre_file,BGR_Hist_image_K_mean):
    R_channel = os.path.join(BGR_signal_file_one_channel, "RGB_channel", "R_channel")
    G_channel = os.path.join(BGR_signal_file_one_channel, "RGB_channel", "G_channel")
    B_channel = os.path.join(BGR_signal_file_one_channel, "RGB_channel", "B_channel")

    if not os.path.exists(BGR_Hist_image_K_mean):
        os.makedirs(BGR_Hist_image_K_mean)

    for name in tqdm.tqdm(os.listdir(R_channel)):
        R_channel_path = os.path.join(R_channel, name)
        G_channel_path = os.path.join(G_channel, name)
        B_channel_path = os.path.join(B_channel, name)
        mask_path = os.path.join(model_pre_file,name)
        save_path = os.path.join(BGR_Hist_image_K_mean,name)
        R_img = cv2.imread(R_channel_path,cv2.IMREAD_GRAYSCALE)
        G_img = cv2.imread(G_channel_path, cv2.IMREAD_GRAYSCALE)
        B_img = cv2.imread(B_channel_path, cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(mask_path,cv2.IMREAD_GRAYSCALE)
        retVal, mask = cv2.threshold(mask, 45, 255, cv2.THRESH_TOZERO)

        R_hist = cv2.calcHist([R_img], [0], mask, [256], [0, 256])
        G_hist = cv2.calcHist([G_img], [0], mask, [256], [0, 256])
        B_hist = cv2.calcHist([B_img], [0], mask, [256], [0, 256])

        plt.figure(figsize=(6,7))
        ax = plt.gca()
        y_major_locator = MultipleLocator(1000)
        ax.yaxis.set_major_locator(y_major_locator)
        plt.xlim(0, 256)
        plt.ylim(0, 9000)
        plt.xlabel('Pixels')
        plt.ylabel('Numbers')

        plt.plot(R_hist, label='red-channel', color='red')
        plt.plot(G_hist, label='green-channel', color='green')
        plt.plot(B_hist, label='blue-channel', color='blue')
        plt.title('day'+str((name.split('_')[0]).split('-')[-1]))
        plt.legend()

        plt.savefig(save_path)
        plt.close('all')

#### 图像叠加分类

In [ ]:
import cv2,glob,os,tqdm
import numpy as np
from PIL import Image

def fun(ground_true,file_list,o):
    if not os.path.exists(o):
        os.makedirs(o)
    for p in tqdm.tqdm(file_list):
        for name in tqdm.tqdm(os.listdir(ground_true)):
            img1_p = os.path.join(ground_true,name)
            img2_P = os.path.join(p,name)
            img1 = cv2.imread(img1_p,cv2.IMREAD_GRAYSCALE)
            img2 = cv2.imread(img2_P,cv2.IMREAD_GRAYSCALE)

            img1 = np.where(img1!=0,1,0)
            img2 = np.where(img2!=0,2,0)

            imgs_union = img1+img2

            imgs_union = Image.fromarray(imgs_union)

            imgs_union = imgs_union.convert('L')
                    #调色板
            palette = [0, 0, 0,0, 255, 0, 255, 0, 0,255,255,255]  #
                        #0 背景 1 标注有，预测没有 2标注没有，预测有 3 两者交集
                    #着色
            imgs_union.putpalette(palette)
            save_path = os.path.join(o,name.split('.')[0]+'_'+p.split('\\')[-1]+'.png')
            imgs_union.save(save_path)

file_list  =glob.glob(r"D:\31890\Desktop\tranformer\model_compare\pred\*")
ground_true = r"D:\31890\Desktop\tranformer\model_compare\label"
o = r"D:\31890\Desktop\tranformer\model_compare\fenlei"
fun(ground_true,file_list,o)

#### 图像的连通域操作

In [ ]:
import cv2
import numpy as np
 
# 读入图片
img = cv2.imread("001.jpg")
# 中值滤波，去噪
img = cv2.medianBlur(img, 3)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.namedWindow('original', cv2.WINDOW_AUTOSIZE)
cv2.imshow('original', gray)
 
# 阈值分割得到二值化图片
ret, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
 
# 膨胀操作
kernel2 = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
bin_clo = cv2.dilate(binary, kernel2, iterations=2)
 
# 连通域分析
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(bin_clo, connectivity=8)
 
# 查看各个返回值
# 连通域数量
print('num_labels = ',num_labels)
# 连通域的信息：对应各个轮廓的x、y、width、height和面积
print('stats = ',stats)
# 连通域的中心点
print('centroids = ',centroids)
# 每一个像素的标签1、2、3.。。，同一个连通域的标签是一致的
print('labels = ',labels)
 
# 不同的连通域赋予不同的颜色
output = np.zeros((img.shape[0], img.shape[1], 3), np.uint8)
for i in range(1, num_labels):
 
    mask = labels == i
    output[:, :, 0][mask] = np.random.randint(0, 255)
    output[:, :, 1][mask] = np.random.randint(0, 255)
    output[:, :, 2][mask] = np.random.randint(0, 255)
cv2.imshow('oginal', output)
cv2.waitKey()
cv2.destroyAllWindows()

## cv像素操作

#### 像素值替换

In [ ]:
def pixel_replacement(image_file,save_file):
    for name in tqdm.tqdm(os.listdir(image_file)):
        save_path = os.path.join(save_file,name)
        image_path = os.path.join(image_file,name)
        img = cv2.imread(image_path)
        img= cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        h,w,c = img.shape
        b = np.empty([h,w])
        for x in range(h):
            for y in range(w):
                #黑色[0,0,0]
                if (img[x,y]==[0,0,0]).all():
                    continue
                #红色[255,0,0]
                elif (img[x,y]==[255,0,0]).all():
                    b[x,y]=2
                #绿色[0,255,0]
                elif (img[x,y,]==[0,255,0]).all():
                    b[x,y]=3
                else:
                    b[x,y]=1
        cv2.imwrite(save_path,b)

def pixel_grayscale(image_file,save_file):
    if not os.path.exists(save_file):
        os.makedirs(save_file)
    for name in tqdm.tqdm(os.listdir(image_file)):
        save_path = os.path.join(save_file,name)
        img = cv2.imread(os.path.join(image_file,name),cv2.IMREAD_GRAYSCALE)
        h,w = img.shape
        b = np.empty([h,w])
        for x in range(h):
            for y in range(w):
                #黑色[0,0,0]
                if (img[x,y]==0).all():
                    continue
                #红色[255,0,0]
                elif (img[x,y]==250).all():
                    b[x,y]=1
                #绿色[0,255,0]
                elif (img[x,y,]==76).all():
                    b[x,y]=2
                else:
                    b[x,y]=3
        cv2.imwrite(save_path,b)
image_file = r'D:\31890\Desktop\codefile\Utils\biaozhuchuli'
save_file  = r'D:\31890\Desktop\codefile\Utils\biaozhu'
pixel_grayscale(image_file,save_file)

#### 像素提取

In [ ]:
def pixel_replacement(image_file,death_file,live_file):
    for name in tqdm.tqdm(os.listdir(image_file)):
        image_path = os.path.join(image_file,name)
        save_death = os.path.join(death_file,name)
        save_live = os.path.join(live_file,name)
        img = cv2.imread(image_path)
        img= cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        h,w,c = img.shape
        #death
        b = np.empty([h,w])
        #live
        b2 = np.empty([h,w])
        for x in range(h):
            for y in range(w):
                #黑色[0,0,0]
                if (img[x,y]==[0,0,0]).all():
                    continue
                #活根
                elif (img[x,y]==[128,0,0]).all():
                    b[x,y]=255
                else:
                    b2[x,y]=255
        cv2.imwrite(save_death,b)
        cv2.imwrite(save_live,b2)
image_file = r'D:\31890\Desktop\senescence\senescence\mix'
death_file  = r'D:\31890\Desktop\senescence\senescence\death'
live_file = r'D:\31890\Desktop\senescence\senescence\live'
pixel_replacement(image_file,death_file,live_file)

#### 阈值操作

In [ ]:
def threshold2(image_array,lower=[0,108,118],upper=[45,188,109]):
    lower_red = np.array(lower)
    upper_red = np.array(upper)
    #lower_red～upper_red之间的值变成255
    mask = cv2.inRange(image_array,lower_red,upper_red)
    mask
    # imgResult = cv2.bitwise_and(image_array,image_array,mask=mask)
    # return cv2.cvtColor(imgResult,cv2.COLOR_HSV2BGR)

#单张图像
# image_path = r'D:\software\code\codefile\T\image\compare_image\splic\result\1.png'
# save_path = r'D:\software\code\codefile\T\image\compare_image\splic\binary\1.png'
# image_array = cv2.imread(image_path)
# result = threshold(image_array, 35, 255)
# cv2.imwrite(save_path,result)

#系列图像
image_file = r"D:\software\code\codefile\T\image\Root_senescence\1_egnet_result\Splicing\mask\2_512-64"
save_file  = r"D:\software\code\codefile\T\image\Root_senescence\1_egnet_result\Splicing\binary\2_512-64"
if not os.path.exists(save_file):
    os.makedirs(save_file)
for name in tqdm.tqdm(os.listdir(image_file)):
    image_path = os.path.join(image_file,name)
    save_path = os.path.join(save_file,name)
    image_array = cv2.imread(image_path)
    result= threshold2(image_array, lower=[0,108,118],upper=[45,188,109])
    cv2.imwrite(save_path,result)


#### 根据阈值转换为黑白图像

In [ ]:
def fun(img_file,save_file,lower):
    save_file = save_file+"_threshold"+str(lower)
    if not os.path.exists(save_file):
        os.makedirs(save_file)
    for name in tqdm.tqdm((os.listdir(img_file))):
        single_image_path = os.path.join(img_file, name)
        image = cv2.imread(single_image_path,cv2.IMREAD_GRAYSCALE)
        imagel = np.where(image>lower,255,0)
        cv2.imwrite(os.path.join(save_file,name),imagel)

i = r'D:\31890\Downloads\2_512-64'
o = r'D:\31890\Downloads\2_512-64-155'
fun(i,o,155)

In [ ]:
def fun(img_file,save_file):
    if not os.path.exists(save_file):
        os.makedirs(save_file)
    for name in tqdm.tqdm((os.listdir(img_file))):
        single_image_path = os.path.join(img_file, name)
        image = cv2.imread(single_image_path,cv2.IMREAD_GRAYSCALE)
        imagel = np.where(image>0,1,0)
        cv2.imwrite(os.path.join(save_file,name),imagel)

i = r'D:\31890\Desktop\my_data\anno\train'
o = r'D:\31890\Desktop\my_data\anno'
fun(i,o)

#### 生成mask图像操作

In [ ]:
import os,tqdm,cv2,sys
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator

#取消像素限制
Image.MAX_IMAGE_PIXELS = None
def Mask(org_img_file,mode_pre_file,out_file,color):
    '''     org_img_file:原始图片
            mode_pre_file:模型预测结果11
            out_file:保存路径
            color:颜色通道   '''
    for name in tqdm.tqdm(os.listdir(org_img_file)):
        image_name = os.path.splitext(name)[0]
        image_path = os.path.join(org_img_file, name)
        mask_path = os.path.join(mode_pre_file, image_name+'.png')
        #cv2读取图片的方式为BGR
      
        image = cv2.imread(image_path)
        
        #模型预测的灰度图片
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        h,w  = mask.shape
        image = cv2.resize(image,(w,h))
        #模型预测图片阈值剪切
        retVal, mask = cv2.threshold(mask, 45, 255, cv2.THRESH_TOZERO)
        mask = np.where(mask==0,255,0).astype(np.uint8)
        # retVal, mask = cv2.threshold(mask, 45, 255, cv2.THRESH_TOZERO)
        #是否颜色空间转换
        if color:
            if color=="BGR":
                if not os.path.exists(os.path.join(out_file,color)):
                    os.makedirs(os.path.join(out_file,color))
                color_mask = os.path.join(out_file,color,image_name+'.png')
                image2 = cv2.bitwise_and(image, image, mask=mask) 
                cv2.imwrite(color_mask, image2)
            elif color=="HSV":
                if not os.path.exists(os.path.join(out_file,color)):
                    os.makedirs(os.path.join(out_file,color))
                color_mask = os.path.join(out_file,color,image_name+'.png')
                image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
                image2 = cv2.bitwise_and(image, image, mask=mask) 
                cv2.imwrite(color_mask, image2)
            elif color=="Lab":
                if not os.path.exists(os.path.join(out_file,color)):
                    os.makedirs(os.path.join(out_file,color))
                color_mask = os.path.join(out_file,color,image_name+'.png')
                image = cv2.cvtColor(image, cv2.COLOR_BGR2Lab)
                image2 = cv2.bitwise_and(image, image, mask=mask) 
                cv2.imwrite(color_mask, image2) 
i = r"D:\31890\Desktop\tranformer\senescence\in-situ2"
m = r"D:\31890\Desktop\codefile\result\Train_data\Datasets\GAN\completion_label"
o = r"D:\31890\Desktop\codefile\result\Train_data\Datasets\GAN\cimage"
Mask(i,m,o,"BGR")

#### 索引图着色

In [ ]:
import os
import tqdm
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import sklearn.cluster 
import pandas as pd
#索引图制作RGB像素图
def colormap(input_file,save_file):
    '''     input_file:索引图输入图片
            save_file:RGB图片的路径   '''
    if not os.path.exists(save_file):
        os.makedirs(save_file)
    for img_name in tqdm.tqdm(os.listdir(input_file)):
        img_path = os.path.join(input_file,img_name)
        img = Image.open(img_path)
        img.convert('L')
        #调色板
        palette = [0, 0, 0,0, 128, 0, 128, 128, 0]
        # palette = [0, 0, 0, 128, 0, 0,0, 128, 0, 128, 128, 0,
        #         0, 0, 128,]
                # 128, 0, 128, 0, 128, 128, 128, 128, 128,
                # 64, 0, 0
        #着色
        img.putpalette(palette)
        img.save(os.path.join(save_file,img_name))
input_File= r'D:\31890\Pictures\image\Root_senescence\clustering_image\2\hsv\s_channel'
save_File  = r'D:\31890\Pictures\image\Root_senescence\clustering_image\2\hsv\s_channel_clustering'
colormap(input_File,save_File)

#### 单通道灰度图提取

In [ ]:

def single_channel(Three_channel_img,color,save_file):
    '''     Three_channel_img:三通道图片
            color:颜色通道
            save_file:保存路径      '''
    #创建保存路径
    save_file_list = []
    for col in color:
        if not os.path.exists(os.path.join(save_file,color,col+'_channel')):
            path = os.makedirs(os.path.join(save_file,color,col+'_channel'))
        save_file_list.append(os.path.join(save_file,color,col+'_channel'))
    
    for name in tqdm.tqdm(os.listdir(Three_channel_img)):
        Tre_img_path = os.path.join(Three_channel_img, name)
        image = cv2.imread(Tre_img_path)
        #通道一
        image1 = image[:, :, 0]
        image1 = image1.astype(np.uint8)
        cv2.imwrite(os.path.join(save_file_list[0], name), image1)
        #通道二
        image2 = image[:, :, 1]
        image2 = image2.astype(np.uint8)
        cv2.imwrite(os.path.join(save_file_list[1], name), image2)
        #通道三
        image3 = image[:, :, 2]
        image3 = image3.astype(np.uint8)
        cv2.imwrite(os.path.join(save_file_list[2], name), image3)

Three_channel_img = r'D:\31890\Pictures\image\Root_senescence\egnet_pre_cv2\color_mask\2'
save_file = r'D:\31890\Pictures\image\Root_senescence\channel_image\2'
single_channel(Three_channel_img,"hsv",save_file)

#### 图像和分割图融合

In [ ]:
# 将分割图和原图合在一起
from PIL import Image
import matplotlib.pyplot as plt
 
def shwo(img_p1,img_p2):
    image1 = Image.open(img_p1)     # 原图
    image2 = Image.open(img_p2)     # 分割图
    
    plt.figure()

    plt.subplot(221)
    plt.imshow(image1)
    
    plt.subplot(222)
    plt.imshow(image2)
    
    plt.subplot(223)
    plt.imshow(image1)
    plt.imshow(image2,alpha=0.5)
    plt.show()

def fuse(img_p1,img_p2,save_p):
    image1 = Image.open(img_p1)#image1 原图 
    image2 = Image.open(img_p2)#image2 分割图

    # image2.convert('L')
    # palette = [0, 0, 0,0, 128, 0, 128, 128, 0,255,255,255]
    # image2.putpalette(palette)
    
    image1 = image1.convert('RGBA')
    image2 = image2.convert('RGBA')
    
    #两幅图像进行合并时，按公式：blended_img = img1 * (1 – alpha) + img2* alpha 进行
    image = Image.blend(image1,image2,0.3)
    image.save(save_p)
    image.show()

def cv_fuse(img_p1,img_p2,save_p):
    imgfile = 'image.jpg'
    pngfile = 'mask.png'

    img = cv2.imread(imgfile, 1)
    mask = cv2.imread(pngfile, 0)

    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(img, contours, -1, (0, 0, 255), 1)

    img = img[:, :, ::-1]
    img[..., 2] = np.where(mask == 1, 255, img[..., 2])

    plt.imshow(img)
    plt.show()

img_p1 = r"D:\software\Code\codefile\Utils\Image\1.tif"
img_p2 = r'D:\software\Code\codefile\Utils\Image\3.tif'
save_p = r'D:\software\Code\codefile\Utils\Image\4.tif'
fuse(img_p1,img_p2,save_p)

## cv2聚类

#### 图像聚类

In [ ]:
BGR = ([225,105,65],[0,255,255],[192,192,192],[255,255,240],[0,97,255],)


def K_means_method_three_dim(image_path,k_means_path,n_class):
    '''     image_path:三通道图片
            k_means_path:聚类结果   '''
            
    img = cv2.imread(image_path,);
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    #展平
    image_flat = img.reshape(-1,3)
    image_flat = np.float32(image_flat)
    #终止条件
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TermCriteria_MAX_ITER, 20, 0.5)
    #flags
    flags = cv2.KMEANS_RANDOM_CENTERS
    #flags = cv2.KMEANS_PP_CENTERS
    #K-Means聚类 聚集成4类
    compactness, labels, centers = cv2.kmeans(image_flat, n_class, None, criteria, 10, flags)
    #labels.shape=((3408500, 1))      centers.shape = ((4, 3))颜色矩阵
    centers = np.uint8(centers)
    for i in range(centers.shape[0]):
        centers[i,:]
        if centers[i,0]==centers[i,1]==centers[i,2]:
                continue
        else:
                centers[i,:] = BGR[i]
#     print(centers)
    res = centers[labels.flatten()]
    save = res.reshape((img.shape))
    cv2.imwrite(k_means_path, save)

image_path =r"D:\software\Code\code-file\pytorch-template\test\cmask_sub.png"
k_means_path = r"D:\software\Code\code-file\pytorch-template\test\cmask_sub_km.png"
K_means_method_three_dim(image_path,k_means_path,n_class=3)


#### 图像的K-means聚类

In [ ]:
# 单通道
def K_means_method_one_dim(image_path,k_means_path):
    '''     image_path:三通道图片
            k_means_path:聚类结果   '''
    img = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
    #展平
    image_flat = img.reshape(-1, 1)
    image_flat = np.float32(image_flat)
    #终止条件
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TermCriteria_MAX_ITER, 20, 0.5)
    #flags
    flags = cv2.KMEANS_RANDOM_CENTERS
    #flags = cv2.KMEANS_PP_CENTERS
    #K-Means聚类 聚集成4类
    compactness, labels, centers = cv2.kmeans(image_flat, 4, None, criteria, 10, flags)
    #labels.shape=((3408500, 1))      centers.shape = ((4, 3))
    centers = np.uint8(centers)
    res = centers[labels.flatten()]
    save = res.reshape((img.shape))
    cv2.imwrite(k_means_path, save)

In [ ]:
# 三通道
def K_means_method_three_dim(image_path,k_means_path):
    '''     image_path:三通道图片
            k_means_path:聚类结果   '''
    img = cv2.imread(image_path)
    #展平
    image_flat = img.reshape(-1,3)
    image_flat = np.float32(image_flat)
    #终止条件
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TermCriteria_MAX_ITER, 20, 0.5)
    #flags
    flags = cv2.KMEANS_RANDOM_CENTERS
    #flags = cv2.KMEANS_PP_CENTERS
    #K-Means聚类 聚集成4类
    compactness, labels, centers = cv2.kmeans(image_flat, 5, None, criteria, 10, flags)
    #labels.shape=((3408500, 1))      centers.shape = ((4, 3))
    centers = np.uint8(centers)
    res = centers[labels.flatten()]
    save = res.reshape((img.shape))
    cv2.imwrite(k_means_path, save)

## cv2超像素

#### cv2

In [ ]:
import os,tqdm,cv2
import numpy as np

def Superpixel_segmentation(image_array):
    #初始化slic项，超像素平均尺寸20（默认为10），平滑因子20
    slic = cv2.ximgproc.createSuperpixelSLIC(image_array,region_size=50,ruler = 20)
    slic.iterate(10)                                    #迭代次数，越大效果越好
    mask_slic = slic.getLabelContourMask()              #获取Mask，超像素边缘Mask==1
    label_slic = slic.getLabels()                       #获取超像素标签矩阵

    cv2.imwrite(r'D:\31890\Desktop\codefile\Utils\Image\3.png',label_slic)

    number_slic = slic.getNumberOfSuperpixels()         #获取超像素数目
    print(number_slic)
    mask_inv_slic = cv2.bitwise_not(mask_slic)


    img_slic = cv2.bitwise_and(image_array,image_array,mask =  mask_inv_slic)  #在原图上绘制超像素边界
    # cv2.imshow("img_slic",img_slic)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return img_slic

def Superpixel_averaging(image_array):
    #初始化slic项，超像素平均尺寸20（默认为10），平滑因子20
    slic = cv2.ximgproc.createSuperpixelSLIC(image_array,region_size=20,ruler = 40.0)
    slic.iterate(10)                                    #迭代次数，越大效果越好
    label_slic = slic.getLabels()                       #获取超像素标签
    number_slic = slic.getNumberOfSuperpixels()         #获取超像素数目
    color_array = np.zeros((number_slic,3))
    for i in range(number_slic):
        temp_array = image_array[label_slic==(i)]
        temp_array_mean = np.mean(temp_array,axis=0)
        color_array[i]=temp_array_mean
    color_array = color_array.astype(np.int32)
    result = color_array[(label_slic).flatten()]
    result = result.reshape(image_array.shape)
    return result

image_array = cv2.imread(r"D:\31890\Desktop\codefile\Utils\Image\2-2019-7-07-1200_8_13_.png")
result = Superpixel_segmentation(image_array)
cv2.imwrite(r"D:\31890\Desktop\codefile\Utils\Image\2.png",result)

#### 衰老超像素后处理

In [ ]:
import cv2,os,tqdm
import numpy as np
from PIL import Image

def shuailao_slic(image_array,mask_array):
    #初始化slic项，超像素平均尺寸20（默认为10），平滑因子20
    slic = cv2.ximgproc.createSuperpixelSLIC(image_array,region_size=20,ruler = 20)
    slic.iterate(50)                                    #迭代次数，越大效果越好

    label_slic = (slic.getLabels() +1).astype(np.uint8)                 #获取超像素标签矩阵
    label_slic = cv2.bitwise_and(label_slic, label_slic, mask=mask_array.astype(np.uint8)).astype(np.uint8)

    class_num = np.unique(label_slic,return_index=False,return_inverse=False)
    for i in range(len(np.unique(label_slic,return_index=False,return_inverse=False))):
        label_slic[np.where(label_slic==class_num[i])] = i  #重新排列分类矩阵

    image_gray = cv2.cvtColor(image_array,cv2.COLOR_BGR2GRAY)
    image_gray = cv2.bitwise_and(image_gray, image_gray, mask=mask_array).astype(np.uint8)

    image_copy = image_gray.copy()

    # a1,b1 = np.unique(image_gray,return_counts=True)
    # print(a1,b1)

    for i in range(len(class_num)):
        a,b = np.unique(image_gray[label_slic==i],return_counts=True)
        # print(a,b)
        # print(a[np.argmax(b)])
        if  a[np.argmax(b)]==0:
            image_copy[np.where(label_slic==i)] = a[np.argmax(b)]
            # a2,b2 = np.unique(image_copy[label_slic==i],return_counts=True)
            # print(a2,b2)
        elif a[np.argmax(b)]==141:
            image_copy[np.where(label_slic==i)] = 1
        elif a[np.argmax(b)]==76:
            image_copy[np.where(label_slic==i)] = 2
        else:
            image_copy[np.where(label_slic==i)] = 3
            

    image_copy = Image.fromarray(image_copy)
    image_copy.convert('L')
    #调色板
    palette = [0, 0, 0,0, 255, 0, 255, 0, 0,255,255,255]
    # palette = [0, 0, 0, 128, 0, 0,0, 128, 0, 128, 128, 0,
    #         0, 0, 128,]
            # 128, 0, 128, 0, 128, 128, 128, 128, 128,
            # 64, 0, 0
    #着色
    image_copy.putpalette(palette)

    return image_copy

def main(i,m,o):
    if not os.path.exists(o):
        os.makedirs(o)
    for name in tqdm.tqdm(os.listdir(i)):
        name  = name.split('.')[0]
        image_array = cv2.imread(os.path.join(i,name+'.png'))
        mask_array  = cv2.imread(os.path.join(m,name+'.png'),cv2.IMREAD_GRAYSCALE)
        image_rebuild = shuailao_slic(image_array,mask_array)
        image_rebuild.save(os.path.join(o,name+'.png'))

i = r"D:\31890\Desktop\tranformer\senescence\shuailao"
m = r"D:\31890\Desktop\tranformer\senescence\mask_heidi"
o = r"D:\31890\Desktop\tranformer\senescence\shuailao_re2"
main(i,m,o)


## 其他

#### 分离主根

In [ ]:
#滤波
def blus(image_array):
    img_blur=cv2.blur(image_array,(8,8))
    return img_blur
#阈值
def threshold(image_array, lower, upper):
    result = np.where(image_array>lower,255,0)
    return result
# Scharr算子实现梯度计算
def Scharr_demo(image_array):
    # x 方向梯度
    image_grad_x = cv2.Scharr(image_array, cv2.CV_32F, 1, 0)
    # 分别求绝对值并转化为8位的图像上，这样做方便显示
    image_gradx = cv2.convertScaleAbs(image_grad_x) 
    return image_gradx
#形态学操作
def morphologyEx_demo(image_array):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,40))#3,40
    result  = cv2.morphologyEx(image_array,cv2.MORPH_OPEN,kernel,iterations=4)
    return result
#图片相减
def math_demo(image1_array,image2_array,add_or_sub="add"):
    if add_or_sub=="add":
        image = image1_array + image2_array
    if add_or_sub=="sub":
        image = image1_array - image2_array
    return image

# 分割主根操作(基于egnet)
#大图分割结果
image1_file = r"D:\31890\Desktop\tranformer\senescence\mask_heidi"
save_file   = r"D:\31890\Desktop\tranformer\senescence\mask_heidi_notap"
if not os.path.exists(save_file):
    os.makedirs(save_file)
for name in tqdm.tqdm(os.listdir(image1_file)):
    image1_path  = os.path.join(image1_file,name)
    # image2_path  = os.path.join(image2_file,name)
    save_path = os.path.join(save_file,name)
    image = cv2.imread(image1_path,cv2.IMREAD_GRAYSCALE)
    binary = threshold(image, 35, 255)
    open_image = morphologyEx_demo(image)
    y_grady = Scharr_demo(open_image)
    image = math_demo(open_image,y_grady,'add')
    blus_image = blus(image)
    result = threshold(blus_image, 35, 255)
    result = math_demo(binary,result,'sub')
    cv2.imwrite(save_path,result)

## 测试


In [ ]:
import os,cv2,tqdm
import numpy as np

def pixel_replacement(image_file,live_file):
    if not os.path.exists(live_file):
        os.makedirs(live_file)
    for name in tqdm.tqdm(os.listdir(image_file)):
        image_path = os.path.join(image_file,name)
        save_live = os.path.join(live_file,name)

        img = cv2.imread(image_path)
        img= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        img = np.where(img==38,255,0)


        cv2.imwrite(save_live,img)
image_file= r"D:\31890\Desktop\tranformer\senescence\hunhe"
live_file = r'D:\31890\Desktop\tranformer\senescence\gray'
pixel_replacement(image_file,live_file)